In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras.backend as K
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, GRU, SimpleRNN
from keras.datasets import imdb
from keras.callbacks import History

In [50]:
train = pd.read_csv('/content/drive/MyDrive/Elbrus/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Elbrus/test.csv')
df = pd.concat([train, test], axis=0)

In [51]:
# df_clean = pd.read_csv('/content/drive/MyDrive/Elbrus/clean_tweets')

In [52]:
# df_x = df_clean[:len(train['target'])].drop('target', axis=1)
# df_y = train['target']
# df_test = df_clean[len(train['target']):]

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
x_train, x_valid, y_train, y_valid = train_test_split(train.text, train.target, test_size=0.3, random_state=1)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((5329,), (2284,), (5329,), (2284,))

In [55]:
history = History()
np.random.seed(42)
max_features = 1000

In [56]:
max_l = 150
# for i in range(len(x_train)):
#     if len(x_train.iloc[i]) > max_l: max_l = len(x_train.iloc[i])

In [57]:
max_l

150

In [58]:
top_k = 850
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k, 
                                                  # lower=False,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ '
                                                  )

tokenizer.fit_on_texts(x_train)

In [59]:
x_train_seqs = tokenizer.texts_to_sequences(x_train)
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_seqs, maxlen=max_l)

In [60]:
x_valid_seqs = tokenizer.texts_to_sequences(x_valid)
x_valid = tf.keras.preprocessing.sequence.pad_sequences(x_valid_seqs, maxlen=max_l)

In [61]:
model = Sequential()

model.add(Embedding(max_features, 32))
model.add(LSTM(25, dropout = 0.3))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy', 'AUC'])

In [62]:
%%time
history = model.fit(x_train, y_train, 
                    batch_size = 256,
                    epochs = 50,
                    validation_data = (x_valid, y_valid),
                    verbose = 1, callbacks=[history]   )

Epoch 1/50
21/21 [==============================] - 3s 54ms/step - loss: 0.6887 - accuracy: 0.5415 - auc: 0.5474 - val_loss: 0.6653 - val_accuracy: 0.5806 - val_auc: 0.7091
Epoch 2/50
21/21 [==============================] - 0s 19ms/step - loss: 0.6627 - accuracy: 0.5799 - auc: 0.6950 - val_loss: 0.6235 - val_accuracy: 0.6721 - val_auc: 0.7419
Epoch 3/50
21/21 [==============================] - 0s 18ms/step - loss: 0.6132 - accuracy: 0.6763 - auc: 0.7421 - val_loss: 0.5646 - val_accuracy: 0.7229 - val_auc: 0.7821
Epoch 4/50
21/21 [==============================] - 0s 17ms/step - loss: 0.5460 - accuracy: 0.7333 - auc: 0.8026 - val_loss: 0.5013 - val_accuracy: 0.7627 - val_auc: 0.8326
Epoch 5/50
21/21 [==============================] - 0s 17ms/step - loss: 0.4787 - accuracy: 0.7896 - auc: 0.8507 - val_loss: 0.4757 - val_accuracy: 0.7754 - val_auc: 0.8438
Epoch 6/50
21/21 [==============================] - 0s 17ms/step - loss: 0.4272 - accuracy: 0.8168 - auc: 0.8789 - val_loss: 0.4644 - v

In [63]:
model.evaluate(x_valid, y_valid, batch_size = 64)

36/36 [==============================] - 0s 6ms/step - loss: 0.6766 - accuracy: 0.7649 - auc: 0.8133


[0.6766003966331482, 0.7648861408233643, 0.8133082985877991]

In [64]:
x_test = test.text

In [65]:
x_test_seqs = tokenizer.texts_to_sequences(x_test)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test_seqs, maxlen=max_l)

In [66]:
y_pred_test = np.round(model.predict(x_test)).astype(int)
y_pred_test

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [0]])

In [67]:
submission = pd.read_csv('/content/drive/MyDrive/Elbrus/sample_submission.csv')
submission['target'] = y_pred_test
submission.to_csv('submission_4.csv', index=False)